In [1]:
print('nyan!')

nyan!


In [4]:
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime

### 定数定義

# 2022-23のプレミアリーグ20クラブ
EPL_CODE = [
    'ブレントフォード','リヴァプール','エヴァートン','ブライトン','レスター','フラム','アーセナル','ニューカッスル',
    'マンチェスター・U','ボーンマス','サウサンプトン','フォレスト','リーズ','ウェストハム','アストン・ヴィラ',
    'ウルヴァーハンプトン','クリスタル・パレス','トッテナム','チェルシー','マンチェスター・C'
]

# 曜日（英語 ⇒ 日本語）
WEEK_DAYS_DICT: dict = MappingProxyType({
        'Mon':'（月）',
        'Tue':'（火）',
        'Wed':'（水）',
        'Thu':'（木）',
        'Fri':'（金）',
        'Sat':'（土）',
        'Sun':'（日）'
})

In [15]:
def scrapeSportsNaviPage(t_yyyymmdd: str, t_en_week_name: str, y_yyyymmdd: str, y_en_week_name: str):
    """
    SportsNaviより引数で指定された開催日・曜日（英語）より、対戦相手および試合日時を取得する関数。
    """
    print('scrapeSportsNaviPage start')
    
    # 曜日の英語 ⇒ 日本語へ変換
    t_ja_week_name = WEEK_DAYS_DICT.get(t_en_week_name)
    y_ja_week_name = WEEK_DAYS_DICT.get(y_en_week_name)
    
    # 検索キーの作成
    t_search_key = str(int(t_yyyymmdd[4:6])) + '/' + str(int(t_yyyymmdd[6:8])) + t_ja_week_name
    y_search_key = str(int(y_yyyymmdd[4:6])) + '/' + str(int(y_yyyymmdd[6:8])) + y_ja_week_name

    # 対象ページ
    url = 'https://soccer.yahoo.co.jp/ws/category/eng'

    # スクレイピング
    top_r = requests.get(url)

    # HTMLを解析する
    top_soup = BeautifulSoup(top_r.content, "html.parser")

    # テーブル情報を取得
    td_list = top_soup.find\
            ('table', class_='sc-tableGame').find_all('td')

    # 日付
    match_datetime_list = []
    match_datetime_list_ans = [] #空文字削除後
    
    # 節
    match_kinds_list = []

    # チーム
    team_name_list = []
    
    for td in td_list:
            # 検索対象の試合日時の場合（00:00～24:00）
            # print(td.text)
            if t_search_key in td.text:
                work_td = td.text.split('\n')
                for name in work_td:
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list.append(name)
                        
            # 検索対象の試合日時の場合（24:01～）
            if y_search_key in td.text:
                work_td = td.text.split('\n')
                for name in work_td:
                    if re.search(r'(\d*):(\d*)', name) is not None:
                        match_datetime_list.append(name)

                        
            # クラブ名を取得
            for num in range(len(EPL_CODE)):
                if EPL_CODE[num] in td.text:
                    team_name_list.append(EPL_CODE[num])
            
    # クラブ名リストの空文字を削除
    team_name_list = [a for a in team_name_list if a != '']    

    print('scrapeSportsNaviPage end')
    
    for name in match_datetime_list:
        #print(name)
        name = name.lstrip()
        #print(name)
        match_datetime_list_ans.append(name)
        
    match_day_list = []
    match_day_list.append(t_search_key)
    match_day_list.append(y_search_key)
        
    return match_day_list, match_datetime_list_ans, team_name_list, ;

In [16]:
def createMatchInfoList(match_info_list):
    """
    SportsNaviより取得したmatch_info_listを加工してTwitterで配信する際の文字列を作成する関数。
    """
    print('createMatchInfoList start')
    
    # ホーム＆アウェーで仕分け
    home_team_name_list = []
    away_team_name_list = []

    # match_info_list[2] = クラブ名
    # 奇数番目はホーム・偶数番目はアウェー
    for num in range(len(match_info_list[2])):
        if num % 2 == 0:
            home_team_name_list.append(match_info_list[2][num])
        elif num % 2 != 0:
            away_team_name_list.append(match_info_list[2][num])
    
    print('away_team_name_list', away_team_name_list)
    print('home_team_name_list', home_team_name_list)
        

    # 試合時間
    match_datetime = []

    for num in range(len(match_info_list[1])):
        match_datetime.append(match_info_list[1][num])
    
    print('createMatchInfoList end')
    
    return match_datetime, home_team_name_list, away_team_name_list

In [17]:
# 現在時刻の取得
today = datetime.datetime.now() 
today_yyyymmdd = today.strftime("%Y%m%d")
today_yyyymmdd_week = today.strftime("%a")

print(today, today_yyyymmdd, today_yyyymmdd_week)

2023-03-01 22:27:09.640781 20230301 Wed


In [18]:
# 操作後の日付取得
yesterday = today + datetime.timedelta(-1)
yesterday_yyyymmdd = yesterday.strftime("%Y%m%d")
yesterday_yyyymmdd_week = yesterday.strftime("%a")

print(yesterday, yesterday_yyyymmdd, yesterday_yyyymmdd_week)

2023-02-28 22:27:09.640781 20230228 Tue


In [19]:
# 直近のプレミアリーグの試合を取得
match_info_list = []
match_info_list = scrapeSportsNaviPage(today_yyyymmdd[0:8], today_yyyymmdd_week, yesterday_yyyymmdd[0:8], yesterday_yyyymmdd_week)

print('match_info_list', match_info_list)

scrapeSportsNaviPage start
scrapeSportsNaviPage end
match_info_list (['3/1（水）', '2/28（火）'], ['28:45', '29:00'], ['アーセナル', 'エヴァートン', 'リヴァプール', 'ウルヴァーハンプトン'])


In [21]:
# １試合ずつ配列を作成する
match_list = createMatchInfoList(match_info_list)

print(match_list)

createMatchInfoList start
away_team_name_list ['エヴァートン', 'ウルヴァーハンプトン']
home_team_name_list ['アーセナル', 'リヴァプール']
createMatchInfoList end
(['28:45', '29:00'], ['アーセナル', 'リヴァプール'], ['エヴァートン', 'ウルヴァーハンプトン'])


In [28]:
# tweet_bot
def _tweetBot(msg : str):
    """
    指定されたメッセージをTwitterする関数。返却値なし　※スレッド機能は後で実施する。parentTweetIDをここで返却する模様。
    """
    # tweetを実施
    tweet_response = client.create_tweet(
        text=msg
    )
    print(tweet_response)

def createTweetMsg(match_list):
    """
    tweetするメッセージを生成する関数。

    Parameters
    ----------
    match_list
        BIG6_listの試合情報
    """
    print('craeteTweetMsg start')
    
    # match_listが空の場合
    if len(match_list) == 0:
        text = '今日はプレミアリーグの試合がありません。'
        _tweetBot(text)
    else:
        # トップテキストを用意
        top_text = '今日はプレミアリーグ󠁧󠁢が '+ str(len(match_list[0])) + ' 試合あるよ！' +'\n' + '絶対に見逃すな！' + '\n\n'

        # 変数を初期化
        text = top_text
        
        # match_listを分解して1行ずつ整理
        for num in range(len(match_list[0])):
            if int(match_list[0][num][0:2]) >= 24:
                match_hour = int(match_list[0][num][0:2]) - 24
                match_hour = str(match_hour).zfill(2)
                print(match_list[0][num][0:2], match_hour)
                match_list[0][num] = str(match_hour) + match_list[0][num][2:5]
                print(match_list[0][num])
            text += match_list[0][num] + ' ' + match_list[1][num] + ' vs ' + match_list[2][num] + '\n\n'

        
        # ツイートする
        print(text)
        _tweetBot(text)
    
    print('craeteTweetMsg end')

In [29]:
MSG = createTweetMsg(match_list)

craeteTweetMsg start
今日はプレミアリーグ󠁧󠁢が 2 試合あるよ！
絶対に見逃すな！

04:45 アーセナル vs エヴァートン

05:00 リヴァプール vs ウルヴァーハンプトン




NameError: name 'client' is not defined